# Evaluate application endpoint using Azure AI Evaluation APIs


## Objective
In this notebook we will demonstrate how to use the target functions with the standard evaluators to evaluate an application endpoint.

This tutorial provides a step-by-step guide on how to evaluate a function

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 20 minutes running this sample. 

## About this example

This example demonstrates evaluating a target function using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [ ]:
%pip install azure-ai-evaluation azure-identity bs4

### Parameters and imports

In [ ]:
import pandas as pd
import os

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator

azure_openai_api_version = "<api version>"
azure_openai_deployment = "<your-deployment>"
azure_openai_endpoint = "<your-endpoint>"

## Target function
We will use a simple `Ask Wiki` application to get answers to questions from wikipedia. We will use `evaluate` API to evaluate `Ask Wiki` applicaton

`Ask Wiki` needs following environment variables to be set

- AZURE_OPENAI_API_VERSION
- AZURE_OPENAI_DEPLOYMENT
- AZURE_OPENAI_ENDPOINT

In [ ]:
# Use the following code to set the environment variables if not already set. If set, you can skip this step.

os.environ["AZURE_OPENAI_API_VERSION"] = azure_openai_api_version
os.environ["AZURE_OPENAI_DEPLOYMENT"] = azure_openai_deployment
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_openai_endpoint

In [ ]:
from askwiki import ask_wiki

ask_wiki(query="What is the capital of India?")

## Data
Reading existing dataset which has bunch of questions we can Ask Wiki

In [ ]:
df = pd.read_json("data.jsonl", lines=True)
print(df.head())

## Configuration
To use Relevance Evaluator, we will Azure Open AI model details that can be passed as model config.


In [ ]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

## Running Relevance Evaluator to understand its input and output

In [ ]:
relevance_evaluator = RelevanceEvaluator(model_config)

relevance_evaluator(
    response="New Delhi is Capital of India",
    context="India is a country in South Asia.",
    query="What is the capital of India?",
)

## Run the evaluation

In [ ]:
results = evaluate(
    data="data.jsonl",
    target=ask_wiki,
    evaluators={
        "relevance": relevance_evaluator,
    },
)

View the results

In [ ]:
pprint(results)

In [ ]:
pd.DataFrame(results["rows"])